In [1]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf


import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns


import requests
import csv
import json

import warnings



from sklearn.linear_model import LinearRegression

In [2]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")

In [3]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [4]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# JP market
# ticker_symbols = [

#     'SMFG'
#     ,'MUFG'
#     ,'TM'
#     ,'MITSY'
# ]

ticker_symbols = [

#     # INDEX
    # 'IVV'
#     ,'QQQ'

#     # consumer staples
    # ,'TGT'
    # ,'STZ'
    # ,'DLTR'
    # ,'DG'
#     ,'TSN'
#     ,'MNST'
#     ,'PG'
#     ,'PM'
#     ,'MO'
    # ,'KR'


#     # finanace
    'TRV'
#     ,'CB'
#     ,'BAC'
    # ,'BRK.B'


      # transportation
    #   ,'CNI'
    #   ,'CP'


    # consumer discre
    # ,'SBUX'


#     # holding stock
    ,'TSM'
    ,'ACN'
    # ,'DHI'
    # ,'STZ'
    ,'GOOG'
    ,'LPG'
    # ,'CE'
    # ,'SOWG'
    # ,'TM'
    ,'TGT'
]

# Consumer Staples
# ticker_symbols = [
#     'MO',
#     'ADM',
#     # 'BF.B', # DATA ISSUE
#     'BG',
#     'CPB',
#     'CHD',
#     'CLX',
#     'KO',
#     'CL',
#     'CAG',
#     'STZ', # !!
#     'COST',
#     'DG',
#     'DLTR', # !!
#     'EL',
#     'GIS',
#     'HSY',
#     'HRL',
#     'K',
#     'KVUE',
#     'KDP',
#     'KMB',
#     'KHC',
#     # 'KR', # DATA ISSUE
#     'LW',
#     # 'MKC', # DATA ISSUE
#     'TAP',
#     'MDLZ',
#     'MNST',
#     'PEP',
#     'PM',
#     'PG',
#     'SJM',
#     'SYY',
#     'TGT', # !!
#     'TSN', # !!
#     'WBA', # DATA ISSUE
#     'WMT'
# ]

# Energy
# ticker_symbols = [
# 'APA',
#  'BKR', # !! HIGH PE, LOW PROFITABILITY, HIGH FORECAST,  BUT CANNOT BEAT FORECAST, 总结与TRGP类似, 基本面更稳妥, 盈利预期高, 有待考证
#  'CVX',
#  'COP',
#  'CTRA',
#  'DVN',
#  'FANG',
#  'EOG',
#  'EQT',
#  'XOM',
#  'HAL', # !!!! 与SLB类似
#  'HES', # !  HIGH PE, UNDERVALUE, 市场炒作高预期, 2024表现强劲, 已经被超热了
#  'KMI',
#  'MRO',
#  'MPC',
#  'OXY',
#  'OKE', # !!
#  'PSX',
#  'SLB', # !!! A BIT HIGH PE, GOOD ROE, MED PROFITABILITY, STEADY GROWTH FORECAST, AND CAN BEAT FORECAST, 总结基本面良好，与HAL类型相似, 市场炒作热度比HAL更高, 值得研究
#  'TRGP', # !！ HGIH PE, HIGH ROE, RELATIVE MORE DEBT THAN EQUITY, LOW PROFITABILITY, HIGH FORECAST, BUT CANNOT BEAT FORECAST, 总结基本面一般, 但是盈利预期非常高, 有待考证
#  'VLO',
#  'WMB'
#  ]


# Consumer Discretionary
# ticker_symbols = [
#  'ABNB',
#  'AMZN',
#  'APTV',
#  'AZO',
#  'BBWI',
#  'BBY',
#  'BKNG',
#  'BWA',
#  'CZR',
#  'KMX',
#  'CCL',
#  'CMG',
#  'DRI',
#  'DECK',
#  'DPZ',
#  'DHI',
#  'EBAY',
#  'ETSY',
#  'EXPE',
#  'F',
#  'GRMN',
#  'GM',
#  'GPC',
#  'HAS',
#  'HLT',
#  'HD',
#  'LVS',
#  'LEN',
#  'LKQ',
#  'LOW',
#  'LULU',
#  'MAR',
#  'MCD',
#  'MGM',
#  'MHK',
#  'NKE',
#  'NCLH',
#  'NVR',
#  'ORLY',
#  'POOL',
#  'PHM',
#  'RL',
#  'ROST',
#  'RCL',
#  'SBUX',
#  'TPR',
#  'TSLA',
#  'TJX',
#  'TSCO',
#  'ULTA',
#  'WYNN',
#  'YUM'
#  ]


# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]

PE_yr_range = 6 # this will return x-1 yr PE range

ticker_dict_json = {}
ticker_dict_pd = {}

In [5]:
def convert_to_numeric_or_zero(value):
    try:
        # Try to convert the string to a float (or int)
        num = round(float(value), 2)
        return num  # Return the original value if it can be converted to a number
    except ValueError:
        # If conversion fails, return '0'
        return 0

# PE TTM Valuation
It is a stock screener partitioned by ticker level

In [28]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):

    print(j, symbol)
    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                stock_split_record_df = pd.DataFrame()
                stock_split_record_df['split_factor'] = 1
                stock_split_record_df['effective_date'] = datetime.today()


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)
            Daily_stock_df.index = pd.to_datetime(Daily_stock_df.index)


    for date_i in Daily_stock_df.index.date:
        for date_j in stock_split_record_df['effective_date'].dt.date:
            if date_i == date_j:

                # stock price to divided the split factor
                Daily_stock_df.loc[Daily_stock_df.index.date < date_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.date == date_j].values[0])


    # MA200 calculation
    Daily_stock_df['MA200'] = Daily_stock_df.sort_index(ascending=True)['stock_price'].rolling(window=200).mean()

    # Calculate the slope of the MA200 using linear regression
    def calculate_slope(series):
        # Create a time index (0, 1, 2, ..., n) for the linear regression
        x = np.arange(len(series)).reshape(-1, 1)
        y = series.values.reshape(-1, 1)

        # Fit the linear regression model
        model = LinearRegression()
        model.fit(x, y)

        # Return the slope
        return model.coef_[0][0]


    # Apply the slope calculation to the MA200 values
    Daily_stock_df['MA200_slope'] = Daily_stock_df.sort_index(ascending=True)['MA200'].rolling(window=20).apply(calculate_slope, raw=False)



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)

    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])



    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - 6) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate metrics
            if Monthly_stock_df.shape[0] <= annualEPS_df.shape[0]: # if Monthly_stock_df has less records than annualEPS_df, choose the mini length record
                annualEPS_df = annualEPS_df[:(Monthly_stock_df.shape[0])]

                annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values
                annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df["PE"].mean().round(2)
                annualEPS_df[f"PE_{PE_yr_range-1}yr_std"] = np.std(annualEPS_df["PE"]).round(2)
                annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] + annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围上限
                annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] - annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围下限



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols
            qtrEPS_df['reportedDate'] = pd.to_datetime(qtrEPS_df['reportedDate'])

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue


    # # forecast 1 qtr earnings from alpha vantage API
    # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

        forecast_earanings_df = pd.DataFrame(
            columns=my_list[0]
            ,data=my_list[1::]
            )
        
        if forecast_earanings_df['estimate'].head(1).values != '':
            latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)
        else:
            latest_projected_EPS = 0


    # forecast 1 year earnings from yf API, forwardPE, PEG
    # yf data
    yf_data = yf.Ticker(symbol).info
    # alpha vintage data
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()
    
    for key, value in data.items():
        if key == 'PEGRatio':
            PEG_12month_projected = convert_to_numeric_or_zero(value=value)
            # if value == 'None':
            #     PEG_12month_projected = 0
            # else:
            #     PEG_12month_projected = round(float(value), 2)

        if key == 'ForwardPE':
           PE_12month_projected = convert_to_numeric_or_zero(value=value)
            # if value == 'None':
            #     PE_12month_projected = 0
            # else:
            #     PE_12month_projected = round(float(value), 2)

    forecast_earnings_keys = [
        'forwardEps'
    ]


    if not all(key in yf_data.keys() for key in forecast_earnings_keys):
        # Handle the case where one or more keys are missing
        EPS_12month_projected = 0
    else:
        EPS_12month_projected = yf_data['forwardEps'] # 代表了截止下一个日历年结束的EPS, next year forecasted EPS


    # US Treasury section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            US_T_10yrs_df = pd.DataFrame(value)
            US_T_10yrs_df['value'] = pd.to_numeric(US_T_10yrs_df['value'], errors='coerce') # change dataframe value to numeric data
            US_T_10yrs_YTM = US_T_10yrs_df['value'][0]



    # COMPANY OVERVIEW
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    
    for key, value in data.items():
        if key  == 'MarketCapitalization':
            stock_mkt_cap = float(value)



    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    

    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')

    
            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])

            qtr_balancesheet_df['BVPS_latest'] = round(
                qtr_balancesheet_df.tail(1)['totalShareholderEquity'].sum() / qtr_balancesheet_df.tail(1)['commonStockSharesOutstanding']
                ,2
                )



    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']





    # Consolidated section
    stock_consolidate_df = Daily_stock_df.head(window_days)


    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)
        past_1_qtr_EPS = filtered_qtrEPS_df.head(1)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()
        EPS_curr_qtr = past_1_qtr_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM
        stock_consolidate_df.loc[i, "EPS_currentQtr"] = EPS_curr_qtr

        if i == max(stock_consolidate_df.index):
            EPS_nextQtr_projected = latest_projected_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        stock_consolidate_df["EPS_nextQtr"] = latest_projected_EPS
        stock_consolidate_df["EPS_nextQtr_TTM"] = EPS_nextQtr_projected


    

    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    stock_consolidate_df["PE_1yr_forward"] = round(PE_12month_projected, 2)
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    stock_consolidate_df["relative_valuation_nextQuater_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_nextQtr_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextQuater_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_nextQtr_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextQuater_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextQuater_projected_+"][0], stock_consolidate_df["relative_valuation_nextQuater_projected_-"][0]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    stock_consolidate_df[f"{window_days}_price_min"] = stock_consolidate_df["stock_price"].min().round(2)
    stock_consolidate_df[f"{window_days}_price_max"] = stock_consolidate_df["stock_price"].max().round(2)
    stock_consolidate_df[f"{window_days}_price_avg"] = stock_consolidate_df["stock_price"].mean().round(2)
    stock_consolidate_df[f"{window_days}_price_std"] = np.std(stock_consolidate_df["stock_price"]).round(2)

    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_std"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_std"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"].values[0]


    stock_consolidate_df["relative_valuation_nextYear_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextYear_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextYear_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextYear_projected_+"][0], stock_consolidate_df["relative_valuation_nextYear_projected_-"][0]])).round(2) #这个是根据next year projected EPS 得出PE估值的价格中位数

    stock_consolidate_df["PEG_next12months"] = PEG_12month_projected
    stock_consolidate_df["EPS_next12months"] = EPS_12month_projected
    stock_consolidate_df["PEG_TTM"] = (stock_consolidate_df["PE_TTM"] / (((EPS_12month_projected - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100)).round(2) # 这个是截止下一年的EPS growth rate所得出的PEG ratio, <1是undervalue的表现
    
    stock_consolidate_df["EPS_nextYr_growthRate"] = (((EPS_12month_projected - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)
    stock_consolidate_df["EPS_nextQtr_growthRate"] = (((stock_consolidate_df["EPS_nextQtr_TTM"] - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)

    stock_consolidate_df["EarningYield_TTM"] = ((stock_consolidate_df["EPS_TTM"] / stock_consolidate_df["stock_price"]) * 100).round(2)
    stock_consolidate_df["ERP_TTM"] = stock_consolidate_df["EarningYield_TTM"] - US_T_10yrs_YTM

    stock_consolidate_df['latest_qtr_liquidation_mktcap_ratio_%'] = stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'].values[-1]
    stock_consolidate_df['BVPS_latest'] = qtr_balancesheet_df['BVPS_latest'].values[-1]

    stock_consolidate_df['MA200_slope'] = Daily_stock_df['MA200_slope']

    # filter conditions
    conditions = [
    (stock_consolidate_df["stock_price"] < stock_consolidate_df["relative_valuation_TTM_-"]),
    (stock_consolidate_df["stock_price"] > stock_consolidate_df["relative_valuation_TTM_+"]),
    ((stock_consolidate_df["stock_price"] >= stock_consolidate_df["relative_valuation_TTM_-"]) & (stock_consolidate_df["stock_price"] <= stock_consolidate_df["relative_valuation_TTM_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    stock_consolidate_df["curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        stock_consolidate_df.loc[condition, "price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    "Ticker"
    ,"stock_price"
    ,"EPS_currentQtr"
    ,"EPS_nextQtr"
    ,"EPS_TTM"
    ,"EPS_nextQtr_TTM"
    ,"EPS_next12months"
    ,"PE_1yr_forward"
    ,"PE_TTM"
    ,"PE_TTM_avg"
    ,"PE_TTM_volatility_+"
    ,"PE_TTM_volatility_-"
    ,f"PE_{PE_yr_range-1}yr_avg"
    ,f"PE_{PE_yr_range-1}yr_volatility_+"
    ,f"PE_{PE_yr_range-1}yr_volatility_-"
    ,"relative_valuation_TTM_+"
    ,"relative_valuation_TTM_-"
    ,"relative_valuation_TTM_median"
    ,"relative_valuation_nextQuater_projected_+"
    ,"relative_valuation_nextQuater_projected_-"
    ,"relative_valuation_nextQuater_projected_median"
    ,"relative_valuation_nextYear_projected_+"
    ,"relative_valuation_nextYear_projected_-"
    ,"relative_valuation_nextYear_projected_median"
    ,"price_valuation_assessment"
    ,"EPS_nextQtr_growthRate"
    ,"EPS_nextYr_growthRate"
    ,"PEG_next12months"
    ,"PEG_TTM"
    ,"EarningYield_TTM"
    ,"ERP_TTM"
    ,"latest_qtr_liquidation_mktcap_ratio_%"
    ,"BVPS_latest"
    ,"MA200_slope"
    ]


    # store each stock info as pd into dictionary
    ticker_dict_pd[symbol] = stock_consolidate_df[selected_cols]
    # transfer pandas dataframe to json format, and each stock info into dictionary
    ticker_dict_json[symbol] = stock_consolidate_df[selected_cols].to_dict()

    if j == 0:
        # screener df creation
        # screener df will store each stock's consolidate df's first row and union them together for screening purposee
        ticker_screen_df = pd.DataFrame(
            columns=selected_cols
        ) 
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]

        # Insert rows into the DataFrame
        ticker_screen_df.loc[j] = stock_consolidate_df_values

    else:
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]
        ticker_screen_df.loc[j] = stock_consolidate_df_values


# ticker screen df consolidated metrics
ticker_screen_df['Industry_PE_TTM_avg'] = round(ticker_screen_df['PE_TTM'].mean(), 2)

0 TRV
1 TSM
2 ACN
3 GOOG
4 LPG
5 TGT


In [30]:
ticker_dict_pd['ACN']

,Ticker,stock_price,EPS_currentQtr,EPS_nextQtr,EPS_TTM,EPS_nextQtr_TTM,EPS_next12months,PE_1yr_forward,PE_TTM,PE_TTM_avg,PE_TTM_volatility_+,PE_TTM_volatility_-,PE_5yr_avg,PE_5yr_volatility_+,PE_5yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,EPS_nextQtr_growthRate,EPS_nextYr_growthRate,PEG_next12months,PEG_TTM,EarningYield_TTM,ERP_TTM,latest_qtr_liquidation_mktcap_ratio_%,BVPS_latest,MA200_slope
2024-09-30,ACN,353.48,2.79,0,11.96,8.69,14.03,27.03,29.56,26.82,28.58,25.06,30.81,38.93,22.69,341.82,299.72,320.77,248.36,217.77,233.06,400.98,351.59,376.28,overvalued,-27.34,17.31,2.27,1.71,3.38,-0.37,2.33,44.15,0.047521
2024-09-27,ACN,349.70,2.79,0,11.96,8.69,14.03,27.03,29.24,26.82,28.58,25.06,30.81,38.93,22.69,341.82,299.72,320.77,248.36,217.77,233.06,400.98,351.59,376.28,overvalued,-27.34,17.31,2.27,1.69,3.42,-0.33,2.33,44.15,0.051024
2024-09-26,ACN,355.81,3.13,0,11.88,8.69,14.03,27.03,29.95,26.82,28.58,25.06,30.81,38.93,22.69,339.53,297.71,320.77,248.36,217.77,233.06,400.98,351.59,376.28,overvalued,-26.85,18.10,2.27,1.65,3.34,-0.41,2.33,44.15,0.055600
2024-09-25,ACN,337.05,3.13,0,11.88,8.69,14.03,27.03,28.37,26.82,28.58,25.06,30.81,38.93,22.69,339.53,297.71,320.77,248.36,217.77,233.06,400.98,351.59,376.28,fair,-26.85,18.10,2.27,1.57,3.52,-0.23,2.33,44.15,0.060488
2024-09-24,ACN,339.62,3.13,0,11.88,8.69,14.03,27.03,28.59,26.82,28.58,25.06,30.81,38.93,22.69,339.53,297.71,320.77,248.36,217.77,233.06,400.98,351.59,376.28,overvalued,-26.85,18.10,2.27,1.58,3.50,-0.25,2.33,44.15,0.066661
2024-09-23,ACN,339.21,3.13,0,11.88,8.69,14.03,27.03,28.55,26.82,28.58,25.06,30.81,38.93,22.69,339.53,297.71,320.77,248.36,217.77,233.06,400.98,351.59,376.28,fair,-26.85,18.10,2.27,1.58,3.50,-0.25,2.33,44.15,0.072728
2024-09-20,ACN,336.22,3.13,0,11.88,8.69,14.03,27.03,28.30,26.82,28.58,25.06,30.81,38.93,22.69,339.53,297.71,320.77,248.36,217.77,233.06,400.98,351.59,376.28,fair,-26.85,18.10,2.27,1.56,3.53,-0.22,2.33,44.15,0.078644
2024-09-19,ACN,335.24,3.13,0,11.88,8.69,14.03,27.03,28.22,26.82,28.58,25.06,30.81,38.93,22.69,339.53,297.71,320.77,248.36,217.77,233.06,400.98,351.59,376.28,fair,-26.85,18.10,2.27,1.56,3.54,-0.21,2.33,44.15,0.084112
2024-09-18,ACN,336.18,3.13,0,11.88,8.69,14.03,27.03,28.30,26.82,28.58,25.06,30.81,38.93,22.69,339.53,297.71,320.77,248.36,217.77,233.06,400.98,351.59,376.28,fair,-26.85,18.10,2.27,1.56,3.53,-0.22,2.33,44.15,0.088499
2024-09-17,ACN,337.04,3.13,0,11.88,8.69,14.03,27.03,28.37,26.82,28.58,25.06,30.81,38.93,22.69,339.53,297.71,320.77,248.36,217.77,233.06,400.98,351.59,376.28,fair,-26.85,18.10,2.27,1.57,3.52,-0.23,2.33,44.15,0.092071


In [59]:
ticker_screen_df = ticker_screen_df.sort_values(by=['nextYear_EPS_growthRate']
                                                ,ascending=False)

selected_cols = [
    'Ticker'
    ,'EPS_TTM'
    ,'PE_TTM'
    ,'PE_TTM_avg'
    ,'price_valuation_assessment'
    ,'nextQuater_EPS_growthRate'
    ,'nextYear_EPS_growthRate'
    ,'next12months_PEG'
    ,'ERP_TTM'
    ,'latest_qtr_liquidation_mktcap_ratio_%'
    ,'Industry_PE_TTM_avg'
]

ticker_screen_df[selected_cols]

,Ticker,EPS_TTM,PE_TTM,PE_TTM_avg,price_valuation_assessment,nextQuater_EPS_growthRate,nextYear_EPS_growthRate,next12months_PEG,ERP_TTM,latest_qtr_liquidation_mktcap_ratio_%,Industry_PE_TTM_avg
1,TSM,5.6,30.8,30.67,fair,8.93,47.68,1.19,-0.43,NaN,18.56
7,CE,9.2,13.38,15.31,undervalued,4.35,38.7,4.42,3.79,-26.59,18.56
0,TRV,16.16,14.72,14.62,fair,11.82,27.78,0.78,3.11,37.34,18.56
5,GOOG,6.97,22.72,25.9,undervalued,4.16,24.96,1.07,0.72,13.97,18.56
4,STZ,12.72,19.86,20.14,fair,3.07,19.18,1.6,1.36,-0.32,18.56
2,ACN,11.88,29.43,26.44,overvalued,0.59,7.91,2.27,-0.28,2.35,18.56
3,DHI,14.89,13.08,10.89,overvalued,-1.88,7.25,0.65,3.96,38.86,18.56
6,LPG,7.64,4.51,5.39,undervalued,-16.88,-25.79,-1.02,18.48,NaN,18.56


In [66]:
ticker_screen_df[ticker_screen_df['price_valuation_assessment'] == 'undervalued']

,Ticker,stock_price,EPS_TTM,EPS_current_qtr,EPS_nextQtr_projected,PE_1yr_forward,PE_TTM,PE_TTM_avg,PE_TTM_volatility_+,PE_TTM_volatility_-,PE_5yr_avg,PE_5yr_volatility_+,PE_5yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,nextQuater_EPS_growthRate,nextYear_EPS_growthRate,next12months_PEG,next12months_EPS,TTM_PEG,EarningYield_TTM,ERP_TTM,latest_qtr_liquidation_mktcap_ratio_%,Industry_PE_TTM_avg
7,CE,123.14,9.2,2.38,9.6,9.16,13.38,15.31,16.7,13.92,11.87,16.13,7.61,153.64,128.06,140.85,160.32,133.63,146.98,213.09,177.62,195.36,undervalued,4.35,38.7,4.42,12.76,0.35,7.47,3.79,-26.59,18.56
5,GOOG,158.37,6.97,1.89,7.26,18.08,22.72,25.9,28.1,23.7,25.05,28.23,21.87,195.86,165.19,180.52,204.01,172.06,188.04,244.75,206.43,225.59,undervalued,4.16,24.96,1.07,8.71,0.91,4.4,0.72,13.97,18.56
6,LPG,34.47,7.64,1.26,6.35,9.96,4.51,5.39,5.92,4.86,1.25,13.24,-10.74,45.23,37.13,41.18,37.59,30.86,34.22,33.57,27.56,30.56,undervalued,-16.88,-25.79,-1.02,5.67,-0.17,22.16,18.48,NaN,18.56


# Financial Statement
it is a stock fundmental dictionary screener partitioned by the time series(yrly, qtrly)

In [60]:
stock_financial_dict = {} # the dict will store ticker as key, the string value of 'annual_df' and 'qtr_df' as nested key, actual json as value

ticker_symbols = [
    'TSM'
    ,'LPG'
    ,'CE'
    ]
# ticker_symbols = electric_utility_symbols
PE_yr_range = 6 # test for 'X' years PE 


# storage process
for j, symbol in enumerate(ticker_symbols):
    print(j, symbol)
    # COMPANY OVERVIEW
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    
    for key, value in data.items():
        if key  == 'MarketCapitalization':
            stock_mkt_cap = float(value)



    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                stock_split_record_df['split_factor'] = 1
                stock_split_record_df['effective_date'] = datetime.today()



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])
    


    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate PE
            annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values




    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    
    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')
                annual_balancesheet_df[f'{column}_YoY'] = annual_balancesheet_df[column].pct_change() * 100

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')
                qtr_balancesheet_df[f'{column}_YoY'] = qtr_balancesheet_df[column].pct_change() * 100

            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])


    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    # CASHFLOW STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_cashflow_df = annual_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_cashflow_df[column] = pd.to_numeric(annual_cashflow_df[column], errors='coerce')
                annual_cashflow_df[f'{column}_YoY'] = annual_cashflow_df[column].pct_change() * 100

            
        if key == 'quarterlyReports':
            qtr_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_cashflow_df = qtr_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_cashflow_df[column] = pd.to_numeric(qtr_cashflow_df[column], errors='coerce')
                qtr_cashflow_df[f'{column}_YoY'] = qtr_cashflow_df[column].pct_change() * 100



    cashflow_ratio_cols = [
        'operatingCashflow'
        ,'capitalExpenditures'
        ,'cashflowFromInvestment'
        ,'cashflowFromFinancing'
        ,'netIncome'
    ]


    ####################
    ### Consolidated ###
    ####################
    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()
    stock_PE_annual_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]
    stock_ratios_annual_consolidate_df[cashflow_ratio_cols] = annual_cashflow_df[cashflow_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]
    stock_ratios_qtr_consolidate_df[cashflow_ratio_cols] = qtr_cashflow_df[cashflow_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    stock_ratios_annual_consolidate_df['PE'] = annualEPS_df["PE"]
    stock_ratios_annual_consolidate_df['reportedEPS'] = annualEPS_df["reportedEPS"]

    # YoY %
    stock_ratios_annual_consolidate_df['QoQ_gross_margin_%_%'] = stock_ratios_annual_consolidate_df["gross_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['QoQ_operating_margin_%_%'] = stock_ratios_annual_consolidate_df["operating_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['QoQ_net_margin_%_%'] = stock_ratios_annual_consolidate_df["net_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['YoY_reportedEPS_%'] = stock_ratios_annual_consolidate_df["reportedEPS"].pct_change() * 100 # yearly EPS 的增速



    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']
    
    # QoQ %
    stock_ratios_qtr_consolidate_df['QoQ_gross_margin_%_%'] = stock_ratios_qtr_consolidate_df["gross_margin_%"].pct_change() * 100
    stock_ratios_qtr_consolidate_df['QoQ_operating_margin_%_%'] = stock_ratios_qtr_consolidate_df["operating_margin_%"].pct_change() * 100
    stock_ratios_qtr_consolidate_df['QoQ_net_margin_%_%'] = stock_ratios_qtr_consolidate_df["net_margin_%"].pct_change() * 100




    # annual PE df
    if stock_PE_annual_consolidate_df.empty:
        stock_PE_annual_consolidate_df['fiscalDateEnding'] = annualEPS_df['fiscalDateEnding']

    stock_PE_annual_consolidate_df[f'{symbol}_PE'] = annualEPS_df["PE"].round(2)



    # reformat the columns order
    stock_ratios_annual_consolidate_df = stock_ratios_annual_consolidate_df[['fiscalDateEnding'] + [col for col in stock_ratios_annual_consolidate_df.columns if col != 'fiscalDateEnding']]
    stock_ratios_qtr_consolidate_df = stock_ratios_qtr_consolidate_df[['fiscalDateEnding'] + [col for col in stock_ratios_qtr_consolidate_df.columns if col != 'fiscalDateEnding']]




    # store the stock, dataframe value pair to the dictionary
    # transfer the pandas df to json
    stock_ratios_annual_consolidate_json = stock_ratios_annual_consolidate_df.to_dict()
    stock_ratios_qtr_consolidate_json = stock_ratios_qtr_consolidate_df.to_dict()

    stock_financial_dict[symbol] = {
    'annual': stock_ratios_annual_consolidate_json
    ,'qtr': stock_ratios_qtr_consolidate_json
    }

0 TSM
1 LPG
2 CE


In [62]:
pd.DataFrame(stock_financial_dict['CE']['annual']).tail(20)

,fiscalDateEnding,gross_margin_%,operating_margin_%,net_margin_%,netIncome,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio,totalShareholderEquity,operatingCashflow,capitalExpenditures,cashflowFromInvestment,cashflowFromFinancing,ROE_%,liquidation_value,liquidation_mktcap_ratio_%,PE,reportedEPS,QoQ_gross_margin_%_%,QoQ_operating_margin_%_%,QoQ_net_margin_%_%,YoY_reportedEPS_%
14,2010-12-31,20.565922,5.946278,10.211195,498000000,1.730220,1.126000e+09,2.721137,7.942765,1.334630,9.260000e+08,596000000,176000000,31000000,-112000000,53.779698,-1.000000e+08,-0.74,NaN,NaN,NaN,NaN,NaN,NaN
13,2011-12-31,20.661881,8.807564,6.601296,377000000,1.951625,1.318000e+09,2.208649,5.351976,1.437545,1.341000e+09,452000000,201000000,-560000000,-388000000,28.113348,3.840000e+08,2.85,NaN,NaN,0.466592,48.118936,-35.352371,NaN
12,2012-11-30,21.673295,6.148669,6.531049,427000000,NaN,NaN,NaN,NaN,NaN,NaN,638000000,349000000,-441000000,-253000000,NaN,NaN,NaN,NaN,NaN,4.895072,-30.188766,-1.064132,NaN
11,2012-12-31,18.977985,2.812148,5.977824,372000000,2.095203,1.484000e+09,2.014825,4.202312,1.570480,1.730000e+09,722000000,361000000,-500000000,49000000,21.502890,7.880000e+08,5.86,NaN,NaN,-12.436087,-54.264113,-8.470692,NaN
10,2013-12-31,19.312394,21.335597,15.577250,1101000000,2.059547,1.637000e+09,1.778253,2.341237,1.539159,2.699000e+09,762000000,370000000,-422000000,-326000000,40.792886,1.759000e+09,13.07,NaN,NaN,1.762089,658.693831,160.583936,NaN
9,2014-12-31,24.425635,11.457074,9.431681,624000000,2.016442,1.360000e+09,1.936029,2.121363,1.431988,2.818000e+09,962000000,254000000,-705000000,-415000000,22.143364,1.937000e+09,14.40,NaN,NaN,26.476474,-46.300665,-39.452207,NaN
8,2015-12-31,23.743470,5.872816,5.476491,304000000,1.798065,1.237000e+09,2.054972,2.610597,1.358065,2.378000e+09,862000000,232000000,-558000000,-66000000,12.783852,1.548000e+09,11.51,NaN,NaN,-2.792825,-48.740701,-41.935156,NaN
7,2016-12-31,26.711027,17.756654,17.110266,900000000,2.295265,1.395000e+09,2.106093,2.229134,1.626741,2.588000e+09,893000000,246000000,-439000000,-759000000,34.775889,1.598000e+09,11.88,NaN,NaN,12.498413,202.353332,212.431209,NaN
6,2017-12-31,25.133067,14.254824,14.021956,843000000,1.790892,1.233000e+09,2.755880,2.303776,1.213598,2.887000e+09,803000000,267000000,-549000000,-351000000,29.199861,1.583000e+09,11.77,NaN,NaN,-5.907521,-19.721228,-18.049457,NaN
5,2018-12-31,27.622916,18.686090,16.907130,1207000000,1.615514,1.095000e+09,3.063927,2.120979,1.027544,2.984000e+09,1558000000,337000000,-507000000,-1165000000,40.449062,1.617000e+09,12.02,8.156845,11.03,9.906667,31.086086,20.576115,NaN


In [63]:
# searching & merge process
YoY_EPS_screener = pd.DataFrame()

for symbol in stock_financial_dict.keys():
    annaul_df = pd.DataFrame(
        stock_financial_dict[symbol]['annual']
    )

    annaul_df = annaul_df[['fiscalDateEnding', 'YoY_reportedEPS_%', 'reportedEPS']].rename(
        columns={
            'YoY_reportedEPS_%': f'{symbol}_YoY_reportedEPS_%'
            ,'reportedEPS': f'{symbol}_reportedEPS'
            }
    )
    annaul_df['fiscalDateEnding'] = pd.to_datetime(annaul_df['fiscalDateEnding']).dt.year


    if YoY_EPS_screener.empty:
        YoY_EPS_screener = annaul_df
    else:
        YoY_EPS_screener = pd.merge(
            YoY_EPS_screener
            ,annaul_df
            ,on='fiscalDateEnding'
            ,how='outer'
        )


In [64]:
YoY_EPS_screener

,fiscalDateEnding,TSM_YoY_reportedEPS_%,TSM_reportedEPS,LPG_YoY_reportedEPS_%,LPG_reportedEPS,CE_YoY_reportedEPS_%,CE_reportedEPS
0,2004,NaN,NaN,NaN,NaN,NaN,NaN
1,2005,NaN,NaN,NaN,NaN,NaN,NaN
2,2006,NaN,NaN,NaN,NaN,NaN,NaN
3,2007,NaN,NaN,NaN,NaN,NaN,NaN
4,2008,NaN,NaN,NaN,NaN,NaN,NaN
5,2009,NaN,NaN,NaN,NaN,NaN,NaN
6,2010,NaN,NaN,NaN,NaN,NaN,NaN
7,2011,NaN,NaN,NaN,NaN,NaN,NaN
8,2012,NaN,NaN,NaN,NaN,NaN,NaN
9,2012,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
stock_PE_annual_consolidate_df

,fiscalDateEnding,FOXA_PE
0,2023,8.45
1,2022,10.89
2,2021,12.86
3,2020,11.74
4,2019,14.48


# SP500 sectors

In [81]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Consumer Staples']['Ticker'].tolist()

In [82]:
# Pivot the DataFrame
pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
pivot_sector_ticker_df = pivot_sector_ticker_df.set_index('Sector').T

pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, I...","[ABNB, AMZN, APTV, AZO, BBWI, BBY, BKNG, BWA, ...","[MO, ADM, BF.B, BG, CPB, CHD, CLX, KO, CL, CAG...","[APA, BKR, CVX, COP, CTRA, DVN, FANG, EOG, EQT...","[AFL, ALL, AXP, AIG, AMP, AON, ACGL, AJG, AIZ,...","[ABT, ABBV, A, ALGN, AMGN, BAX, BDX, BIO, TECH...","[MMM, AOS, ALLE, AAL, AME, ADP, AXON, BA, BR, ...","[ACN, ADBE, AMD, AKAM, APH, ADI, ANSS, AAPL, A...","[APD, ALB, AMCR, AVY, BALL, CE, CF, CTVA, DOW,...","[ARE, AMT, AVB, BXP, CPT, CBRE, CSGP, CCI, DLR...","[AES, LNT, AEE, AEP, AWK, ATO, CNP, CMS, ED, C..."


In [83]:
pivot_sector_ticker_df['Consumer Discretionary'].iloc[0]

['ABNB',
 'AMZN',
 'APTV',
 'AZO',
 'BBWI',
 'BBY',
 'BKNG',
 'BWA',
 'CZR',
 'KMX',
 'CCL',
 'CMG',
 'DRI',
 'DECK',
 'DPZ',
 'DHI',
 'EBAY',
 'ETSY',
 'EXPE',
 'F',
 'GRMN',
 'GM',
 'GPC',
 'HAS',
 'HLT',
 'HD',
 'LVS',
 'LEN',
 'LKQ',
 'LOW',
 'LULU',
 'MAR',
 'MCD',
 'MGM',
 'MHK',
 'NKE',
 'NCLH',
 'NVR',
 'ORLY',
 'POOL',
 'PHM',
 'RL',
 'ROST',
 'RCL',
 'SBUX',
 'TPR',
 'TSLA',
 'TJX',
 'TSCO',
 'ULTA',
 'WYNN',
 'YUM']

In [16]:
len(pivot_sector_ticker_df['Financials'].iloc[0])

71

## Appendix

In [131]:
# Consumer Staples

ticker_symbols = [
    'MO',
    'ADM',
    # 'BF.B', # DATA ISSUE
    'BG',
    'CPB',
    'CHD',
    'CLX',
    'KO',
    'CL',
    'CAG',
    'STZ', # !!
    'COST',
    'DG',
    'DLTR', # !!
    'EL',
    'GIS',
    'HSY',
    'HRL',
    'K',
    'KVUE',
    'KDP',
    'KMB',
    'KHC',
    # 'KR', # DATA ISSUE
    'LW',
    # 'MKC', # DATA ISSUE
    'TAP',
    'MDLZ',
    'MNST',
    'PEP',
    'PM',
    'PG',
    'SJM',
    'SYY',
    'TGT', # !!
    'TSN', # !!
    # 'WBA', # DATA ISSUE
    'WMT'
]

In [132]:
# Energy

ticker_symbols = [
'APA',
 'BKR', # !! HIGH PE, LOW PROFITABILITY, HIGH FORECAST,  BUT CANNOT BEAT FORECAST, 总结与TRGP类似, 基本面更稳妥, 盈利预期高, 有待考证
 'CVX',
 'COP',
 'CTRA',
 'DVN',
 'FANG',
 'EOG',
 'EQT',
 'XOM',
 'HAL', # !!!! 与SLB类似
 'HES', # !  HIGH PE, UNDERVALUE, 市场炒作高预期, 2024表现强劲, 已经被超热了
 'KMI',
 'MRO',
 'MPC',
 'OXY',
 'OKE', # !!
 'PSX',
 'SLB', # !!! A BIT HIGH PE, GOOD ROE, MED PROFITABILITY, STEADY GROWTH FORECAST, AND CAN BEAT FORECAST, 总结基本面良好，与HAL类型相似, 市场炒作热度比HAL更高, 值得研究
 'TRGP', # !！ HGIH PE, HIGH ROE, RELATIVE MORE DEBT THAN EQUITY, LOW PROFITABILITY, HIGH FORECAST, BUT CANNOT BEAT FORECAST, 总结基本面一般, 但是盈利预期非常高, 有待考证
 'VLO',
 'WMB'
 ]

In [135]:
electric_utility_symbols = [
'CEPU'
,'OTTR'
,'PAM'
,'SO' ##
,'DUK'
,'NRG'
,'IDA'
,'DTE' #
,'CEG' ##
,'SRE'
,'ETR'
,'PEG'
,'MGEE'
,'LNT'
,'BKH'
]